In [1]:
import os

import datetime

import numpy as np
import pandas as pd

from pandas import Series
from pandas import DataFrame

from scipy.stats import poisson

import matplotlib.pyplot as plt

In [2]:
downweight_granularity=['FltNum','dtime','Direction','dday','month']

In [3]:
#Data=pd.read_csv('~/Data/Intermediate_Output/R_Output_Test_Pax.csv',sep=',')
Data=pd.read_csv('R_Output_Test_Pax.csv',sep=',')
Data=Data[['DepDate','FltNum','yday','dtime','Direction','month','dday','Dprio','NumPax']]

In [4]:
Map_Dates=DataFrame([Data['DepDate'].unique().tolist(),[pd.to_datetime(x) for x in Data['DepDate'].unique().tolist()]]).transpose()
Map_Dates.columns=['DepDate','DepDate_dt']
Map_Dates['DepDate']=Map_Dates['DepDate'].astype('str')

Data=Data.merge(Map_Dates,on=['DepDate'])
Data['DepDate']=Data['DepDate_dt']
Data=Data[[x for x in Data.columns if x!='DepDate_dt']]

Data['TicketDate']=Data['DepDate']-Data['Dprio'].apply(lambda x: datetime.timedelta(x))

Data=Data.loc[Data['TicketDate']<=pd.to_datetime(datetime.datetime.today().strftime('%Y-%m-%d')),:]

Data=Data[[x for x in Data.columns if x!='TicketDate']]

Data['Dprio']=-Data['Dprio']
Data=Data.set_index(['DepDate','FltNum','yday','dtime','Direction','month','dday','Dprio'])
Data=Data.groupby(level=[0,1,2,3,4,5,6,7]).sum().groupby(level=[0,1,2,3,4,5,6])['NumPax'].cumsum().reset_index()

Data['Dprio']=-Data['Dprio']

In [5]:
#Probability=pd.read_csv('~/Data/Intermediate_Output/Probability.csv')
Probability=pd.read_csv('Probability.csv')

In [6]:
Data=Data.merge(Probability,on=['Dprio']+downweight_granularity)
Data['SpoilageRisk']=poisson.cdf(Data['Cap_downweighted']-Data['NumPax'],Data['Ideal_intermediate_downweighted'])
Data['SpillageRisk']=1-poisson.cdf(Data['Cap_full']-Data['NumPax'],Data['Ideal_intermediate_full'])

In [7]:
Data['Intensity_full']=Data['NumPax']-Data['Ideal_average_full']
Data['Intensity_downweighted']=Data['NumPax']-Data['Ideal_average_downweighted']

In [8]:
deptime=[str(x/60).split('.')[0]+':'+str(round(float('0.'+str(x/60).split('.')[1])*60)) for x in Data['dtime'].unique()]
deptime=[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]
deptime=['0'+x if len(x.split(':')[0])==1 else x for x in deptime]

Map_DepTime=DataFrame([Data['dtime'].unique(),[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]]).transpose()
Map_DepTime.columns=['dtime','deptime']
Map_DepTime['dtime']=Map_DepTime['dtime'].astype('int')

Data=Data.merge(Map_DepTime,on='dtime')

Data['dtime']=Data['deptime']
Data=Data[[x for x in Data.columns if x!='deptime']]

In [9]:
Data['SpoilageRisk']=Data['SpoilageRisk'].apply(lambda x: round(x,3))
Data['SpillageRisk']=Data['SpillageRisk'].apply(lambda x: round(x,3))
Data['Intensity_full']=Data['Intensity_full'].apply(lambda x: round(x,3))
Data['Intensity_downweighted']=Data['Intensity_downweighted'].apply(lambda x: round(x,3))

In [10]:
# Data[['DepDate',
# 'FltNum',
# 'yday',
# 'dtime',
# 'Direction',
# 'month',
# 'dday',
# 'Dprio',
# 'NumPax',
# 'Cap_full',
# 'Cap_downweighted',
# 'Ideal_average_full',
# 'Ideal_average_downweighted',
# 'SpoilageRisk',
# 'SpillageRisk',
# 'Intensity_full',
# 'Intensity_downweighted']].to_csv('~/Data/FrontEnd_Input/GUI_in2.csv',index=False)

Data[['DepDate',
'FltNum',
'yday',
'dtime',
'Direction',
'month',
'dday',
'Dprio',
'NumPax',
'Cap_full',
'Cap_downweighted',
'Ideal_average_full',
'Ideal_average_downweighted',
'SpoilageRisk',
'SpillageRisk',
'Intensity_full',
'Intensity_downweighted']].to_csv('GUI_in2.csv',index=False)